In [1]:
# ignore warnings in the output
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI
api_key = os.getenv("GEMINI_API_KEY")
llm = ChatGoogleGenerativeAI(model="gemini-pro" , api_key=api_key)

In [3]:
from langchain.vectorstores import Chroma
from langchain_core.tools import tool

In [4]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

embedding_function = HuggingFaceBgeEmbeddings(
    model_name = "BAAI/bge-large-en-v1.5",
    model_kwargs = {'device':'cpu'},
    encode_kwargs = {'normalize_embeddings':True}
)

/tmp/ipykernel_349902/175634225.py:3: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceBgeEmbeddings(


In [5]:
ai_assistant_prompt = """
You are an AI assistant that responds strictly with a single word (use_rag, use_sql use_nth) based on the following rules:
    * Respond with use_rag if the prompt concerns the rules, regulations,services provided by Tribhuvan International Airport or any thing_related to Airports.
    * Respond with use_sql if the prompt relates to booking of flight tickets.
    * Respond with use_nth if the prompt is unrelated to Tribhuvan International Airport or if it is a general-purpose query (e.g., greetings, casual conversation).
Your response must always be a single word: (use_rag or use_sql or use_nth). No additional text or explanation is allowed.
"""


In [6]:
rag_prompt = """
You are an advanced AI assistant capable of answering queries based on the provided context. Use the retrieved context to generate accurate and concise responses. Follow these rules:

1. Only use the retrieved context to answer the query. Do not make assumptions or add information not found in the context.
2. If the retrieved context does not contain enough information to answer the query, respond with: "I'm sorry, the context does not provide enough information to answer this question."
3. First summarize the retrieved context, then answer the query.

### Retrieved Context:
{retrieved_context}

### Query:
{query}

### Response:
"""


In [7]:
sql_agent_prompt = """
You are an intelligent SQL and Pandas agent. Your task is to respond to user queries by providing Python Pandas code that retrieves the required information from a CSV file named 'Nepali_Airlines_Data.csv'. Follow these guidelines:

1. Use Pandas to load the CSV file.
2. Write code that filters the data based on the query's requirements.
3. Return only the Pandas code, ensuring it's concise and functional.

My CSV file has the following columns and is structured like this 
    Airline Name,From,To,Estimated Time of Take Off,Estimated Time of Arrival,Type,Death Rate (%)
    Nepal Airlines,Kathmandu,New Delhi,08:00,09:30,International,0.01
    Buddha Air,Kathmandu,Pokhara,06:30,07:15,Domestic,0.005
    Yeti Airlines,Kathmandu,Lukla,07:00,07:30,Domestic,0.02


### Example:
Query: "I want to book a ticket from Chitwan to Lumbini."
Response:
```python
import pandas as pd

# Load the CSV file
df = pd.read_csv("Nepali_Airlines_Data.csv")

# Filter for flights from Chitwan to Lumbini
result = df[(df['From'] == 'Chitwan') & (df['To'] == 'Lumbini')]
print(result)

Input Query: "{query}"

Output Code:
"""

In [8]:
def sql_agent_prompt(query):
    query = query
    sql_agent_prompt = """
        You are an intelligent SQL agent specializing in generating concise SQL queries to retrieve flight information from a database named 'Nepali_Airlines_Data.db'. Your role is to interpret the user's query and produce an accurate, functional SQL query. 

        ### Guidelines:
        1. Assume the database table is named `Nepali_Airlines_Data` with the following columns:
        - `Airline Name`: Name of the airline.
        - `From`: Departure location.
        - `To`: Arrival location.
        - `EstimatedTakeOffTime`: Scheduled departure time.
        - `EstimatedArrivalTime`: Scheduled arrival time.
        - `Type`: Indicates if the flight is "Domestic" or "International".
        - `DeathRate`: Death rate percentage.

        2. Understand the user's requirements and generate an appropriate SQL query.
        3. Return **only** the SQL query as output, enclosed in double quotes (`"`).
        4. Ensure the query is correct, concise, and formatted properly.

        ### Example Query:
        User Query: "I want to book a ticket from Chitwan to Lumbini."
        Response:
        SELECT * FROM Nepali_Airlines_Data WHERE `From` = 'Chitwan' AND `To` = 'Lumbini'

        Your Task:
        1. Read the user query provided as input.
        2. Interpret the query requirements and identify relevant filtering conditions.
        3. Generate a valid SQL query to retrieve the required information.
        4. The generated SQL query should include all the informations about the flight from the database.

        Input Query: "{query}"

        Output Query: """
            
    return sql_agent_prompt.format(query=query)


In [9]:
sql_prompt = """
### Prompt for the LLM:
You are an intelligent assistant specializing in processing and analyzing flight data. Below is the retrieved flight information and the user query. Your task is to interpret the user's request, analyze the flight data, and provide a concise and accurate response.

### Retrieved Flight Information:
{flight_data}

### User Query:
{user_query}

### Guidelines:
1. Use the flight data provided to answer the query.
2. If multiple flights match the query, list them all.
3. If no flights match, clearly state so.
4. Provide additional helpful details, such as estimated times, types (Domestic or International), or death rates if relevant to the query.
5. Format your response in a user-friendly and readable way.
6. Give answer like this:
    * If there is a flight from Kathmandu to Pokhara, then the answer should be: "There is a flight from Kathmandu to Pokhara." then give the rest of the details.
    * If there are multiple flights from Kathmandu to Pokhara, then the answer should be: "There are multiple flights from Kathmandu to Pokhara."then give the rest of the details.
    * If there are no flights from Kathmandu to Pokhara, then the answer should be: "There are no flights from Kathmandu to Pokhara."
"""

In [10]:
def ai_assistant(ai_assistant_prompt,query, llm):
    llm_query = ai_assistant_prompt + query
    # print (llm.invoke(llm_query).content)
    return llm.invoke(llm_query).content

In [11]:
# ai_assistant(ai_assistant_prompt, "What is Gratis Visa?", llm)

In [12]:
from langchain.retrievers.multi_query import MultiQueryRetriever
def rag_assistant(rag_prompt , query, llm):
    db = Chroma(
    persist_directory="../output/Airpott.db",
    embedding_function=embedding_function
)
    retriever = db.as_retriever()
    from langchain.retrievers.document_compressors import EmbeddingsFilter
    from langchain.retrievers import ContextualCompressionRetriever


# using similarity threshold of 0.6
    embeddings_filter  = EmbeddingsFilter(embeddings=embedding_function, similarity_threshold=0.6)
    compression_retriever = ContextualCompressionRetriever(base_compressor=embeddings_filter, base_retriever=retriever)
    compressed_docs = compression_retriever.get_relevant_documents(query = query)
    unique_answers = {i.metadata['answer'] for i in compressed_docs}
    final_docs = "\n".join(unique_answers) 
    final_rag_prompt = rag_prompt.format(retrieved_context = final_docs, query = query)
    # print(final_rag_prompt)
    print(llm.invoke(final_rag_prompt).content)
    return llm.invoke(final_rag_prompt).content

In [13]:
# rag_assistant(rag_prompt, "What are the things that i am not permitted to carry?", llm)

In [14]:
import sqlite3
import pandas as pd
def sql_items_retrieval(sql_agent_prompt, query, llm , sql_prompt):
    llm_query = sql_agent_prompt(query)
    sql_query = llm.invoke(llm_query).content.replace('''"''', "")
    print(sql_query)
    conn = sqlite3.connect("../DB/Nepali_Airlines_Data.db")
    df = pd.read_sql_query(sql_query, conn)
    final_sql_query = sql_prompt.format(flight_data = df.to_string(), user_query = query)
    # print(final_sql_query)
    return llm.invoke(final_sql_query).content

In [15]:
# sql_items_retrieval(sql_agent_prompt, "GIve me the list of Airlines flying form Kathmandu with their full details ", llm , sql_prompt)
# sql_items_retrieval(sql_agent_prompt, "GIve me the list of Airlines flying form Kathmandu to New Dehli.", llm , sql_prompt)

In [16]:
def normal_query(query, llm):
    return llm.invoke(query).content
normal_query("What is Gratis Visa?", llm)

'Gratis Visa is not a real type of visa.'

In [19]:
user_query = "Give me the list of Airlines flying from Kathmandu to New Delhi"
decision = ai_assistant(ai_assistant_prompt, user_query, llm)

if decision == "use_rag":
    print(rag_assistant(rag_prompt, user_query, llm))
elif decision == "use_sql":
    print(sql_items_retrieval(sql_agent_prompt, user_query, llm, sql_prompt))
else:
    print(normal_query(user_query, llm))


SELECT * FROM Nepali_Airlines_Data WHERE `From` = 'Kathmandu' AND `To` = 'New Delhi'
There is a flight from Kathmandu to New Delhi operated by Nepal Airlines. 
- Estimated Time of Take Off: 08:00
- Estimated Time of Arrival: 09:30
- Type: International
- Death Rate: 0.01%
